<h2 id="ref1">REST APIs</h2>


Rest API’s function by sending a request,  the request is communicated via HTTP message. The HTTP message usually contains a JSON file. This contains instructions for what operation we would like the service or resource to perform. In a similar manner, API returns a response, via an HTTP message, this response is usually contained within a JSON.

In cryptocurrency a popular method to display the movements of the price of a currency.

![Example Candlestick Graph](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0101EN-SkillsNetwork/labs/Module%205/candlestick.png)

Here is a description of the candle sticks.

<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0101EN-SkillsNetwork/labs/Module%205/explanation.png" alt="Candle Stick Definition" width="800" height="600">

In this lab, we will be using the \<a href=[https://www.coingecko.com/en/api?utm_medium=Exinfluencer\&utm_source=Exinfluencer\&utm_content=000026UJ\&utm_term=10006555\&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0101ENSkillsNetwork19487395-2021-01-01>CoinGecko](https://www.coingecko.com/en/api?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0101ENSkillsNetwork19487395-2021-01-01%3ECoinGecko) API</a> to create one of these candlestick graphs for Bitcoin. We will use the API to get the price data for 30 days with 24 observation per day, 1 per hour. We will find the max, min, open, and close price per day meaning we will have 30 candlesticks and use that to generate the candlestick graph. Although we are using the CoinGecko API we will use a Python client/wrapper for the API called \<a href=[https://github.com/man-c/pycoingecko?utm_medium=Exinfluencer\&utm_source=Exinfluencer\&utm_content=000026UJ\&utm_term=10006555\&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0101ENSkillsNetwork19487395-2021-01-01>PyCoinGecko](https://github.com/man-c/pycoingecko?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0101ENSkillsNetwork19487395-2021-01-01>PyCoinGecko)</a>. PyCoinGecko will make performing the requests easy and it will deal with the enpoint targeting.


Lets start off by getting the data we need. Using the <code>get_coin_market_chart_by_id(id, vs_currency, days)</code>. <code>id</code> is the name of the coin you want, <code>vs_currency</code> is the currency you want the price in, and <code>days</code> is how many days back from today you want.


In [1]:
!pip3 install pycoingecko
!pip3 install plotly
!pip3 install mplfinance

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [2]:
from pycoingecko import CoinGeckoAPI

In [3]:
cg = CoinGeckoAPI()

bitcoin_data = cg.get_coin_market_chart_by_id(id='bitcoin', vs_currency='usd', days=30)

In [4]:
type(bitcoin_data )

dict

The response we get is in the form of a JSON which includes the price, market caps, and total volumes along with timestamps for each observation. We are focused on the prices so we will select that data.


In [5]:
bitcoin_price_data = bitcoin_data['prices']

bitcoin_price_data[0:5]

[[1653991371161, 31545.550469514776],
 [1653994907736, 31650.838373027826],
 [1653998713268, 31785.404870081464],
 [1654002165332, 31576.510505366634],
 [1654005770730, 31484.74810010492]]

Finally lets turn this data into a Pandas DataFrame.


In [6]:
import pandas as pd

In [7]:
data = pd.DataFrame(bitcoin_price_data, columns=['TimeStamp', 'Price'])

In [8]:
data

,TimeStamp,Price
0,1653991371161,31545.550470
1,1653994907736,31650.838373
2,1653998713268,31785.404870
3,1654002165332,31576.510505
4,1654005770730,31484.748100
...,...,...
716,1656568903339,20002.561151
717,1656572490356,19488.806638
718,1656576061665,19431.122323
719,1656579717360,19262.999869


Now that we have the DataFrame we will convert the timestamp to datetime and save it as a column called <code>Date</code>. We will map our <code>unix_to_datetime</code> to each timestamp and convert it to a readable datetime.


In [9]:
import datetime

In [10]:
data['date'] = data['TimeStamp'].apply(lambda d: datetime.date.fromtimestamp(d/1000.0))


In [11]:
data

,TimeStamp,Price,date
0,1653991371161,31545.550470,2022-05-31
1,1653994907736,31650.838373,2022-05-31
2,1653998713268,31785.404870,2022-05-31
3,1654002165332,31576.510505,2022-05-31
4,1654005770730,31484.748100,2022-05-31
...,...,...,...
716,1656568903339,20002.561151,2022-06-30
717,1656572490356,19488.806638,2022-06-30
718,1656576061665,19431.122323,2022-06-30
719,1656579717360,19262.999869,2022-06-30


Using this modified dataset we can now group by the  <code> Date </code> and find the  <code>  min, max, open, and close </code> for the candlesticks.


In [12]:
candlestick_data = data.groupby(data.date, as_index=False).agg({"Price": ['min', 'max', 'first', 'last']})
candlestick_data

date         Price                                          
                         min           max         first          last
0   2022-05-31  31484.748100  32206.261554  31545.550470  32061.905172
1   2022-06-01  30155.914519  31962.818221  31662.023764  30155.914519
2   2022-06-02  29676.064111  30287.655573  30273.275884  30225.458436
3   2022-06-03  29479.742840  30607.041609  30214.311414  29532.453293
4   2022-06-04  29562.090991  29844.875625  29562.090991  29737.836954
5   2022-06-05  29658.516890  30052.439250  29804.597579  30052.439250
6   2022-06-06  29917.764052  31565.665055  30037.966390  31303.711166
7   2022-06-07  29419.159655  31518.883275  31369.819439  29949.272724
8   2022-06-08  30189.236348  31522.648011  30192.309326  30416.597139
9   2022-06-09  30137.049786  30569.690067  30250.167537  30295.316126
10  2022-06-10  29022.074454  30184.598134  30184.598134  29109.488276
11  2022-06-11  28372.280420  29376.674692  29025.135190  28568.446321
12  2022-06-12  27330.675422  28669.447248  28669.447248  28182.618441
13  2022-06-13  22839.235438  27972.056742  27972.056742  23488.252498
14  2022-06-14  21046.945034  23434.873938  23434.873938  22404.604416
15  2022-06-15  20216.090126  22280.526327  22280.526327  21102.219265
16  2022-06-16  21015.205044  22664.041895  21641.444231  21115.184820
17  2022-06-17  20322.139665  21102.458082  21076.102447  20588.619598
18  2022-06-18  18724.441099  20658.033341  20505.509815  18724.441099
19  2022-06-19  17760.773382  19811.951050  18303.133175  19464.589086
20  2022-06-20  19812.971611  20887.528591  19831.747225  20017.854755
21  2022-06-21  20038.664491  21597.009838  20038.664491  21241.718586
22  2022-06-22  20101.869377  21129.432938  21129.432938  20214.153225
23  2022-06-23  19864.169914  20697.659057  20216.965739  20302.093254
24  2022-06-24  20423.787793  21287.049700  20423.787793  20994.231867
25  2022-06-25  21052.234957  21501.718372  21136.461040  21127.608789
26  2022-06-26  21228.723195  21637.931949  21266.317156  21289.501202
27  2022-06-27  20741.872124  21475.850907  21236.769754  20741.872124
28  2022-06-28  20624.956971  21108.771855  20828.881767  20635.744725
29  2022-06-29  19958.857850  20398.530481  20359.571485  20039.420854
30  2022-06-30  19077.188090  20323.323486  20087.145651  19077.188090

In [13]:
import plotly.graph_objects as go
from plotly.offline import plot
from mplfinance.original_flavor import candlestick2_ohlc

In [14]:
fig = go.Figure(data=[go.Candlestick(x=candlestick_data['date'],
                open=candlestick_data['Price']['first'], 
                high=candlestick_data['Price']['max'],
                low=candlestick_data['Price']['min'], 
                close=candlestick_data['Price']['last'])
                ])

fig.update_layout(xaxis_rangeslider_visible=False)

fig.show()